## Lecture 2: Review of Vectors, Matrices, and Linear Least-Squares Problems


In [33]:
## MP 574 Lecture 2: Review of vectors, matrices, and LLS
##
%matplotlib inline
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
from IPython.display import display, Image
import matplotlib.image as mpimg
from os.path import dirname, join as pjoin
import scipy.io as sio
import scipy.linalg as slinalg

## Linear problems (aka linear systems of equations)

In this course, we will encounter multiple instances of linear problems (ie: linear systems of equations):
\begin{equation}
    \mathbf{A} \mathbf{x} = \mathbf{y}
\end{equation}
Note that these linear systems need not have an exact solution. For instance, does the following linear system (where $\mathbf{x}$ is a length-1 vector) have a solution?
\begin{equation}
      \left[ {\begin{array}{c}
  1  \\
  1  \\
  \end{array} } \right] 
  \mathbf{x} = 
        \left[ {\begin{array}{c}
  1  \\
  7  \\
  \end{array} } \right] 
\end{equation}
(Why or why not?)

How about the following linear system (where $\mathbf{x}$ is a length-2 vector): does it have a solution?
\begin{equation}
      \left[ {\begin{array}{cc}
  1 & 1 \\
  3 & 3 \\
  \end{array} } \right] 
  \mathbf{x} = 
        \left[ {\begin{array}{c}
  1  \\
  1  \\
  \end{array} } \right] 
\end{equation}
(Why or why not?)

How about the following linear system (where $\mathbf{x}$ is a length-2 vector): does it have a solution?
\begin{equation}
      \left[ {\begin{array}{cc}
  1 & 0 \\
  2 & 1 \\
  \end{array} } \right] 
  \mathbf{x} = 
        \left[ {\begin{array}{c}
  1  \\
  1  \\
  \end{array} } \right] 
\end{equation}
(Why or why not?)

In addition, systems of equations oftentimes have multiple solutions. For instance, consider the system of equations (where in this case $\mathbf{x}$ is a length-3 vector): 
\begin{equation}
      \left[ {\begin{array}{ccc}
  1 & 0 & 1 \\
  2 & 1 & 1 \\
  \end{array} } \right] 
  \mathbf{x} = 
        \left[ {\begin{array}{c}
  1  \\
  1  \\
  \end{array} } \right] 
\end{equation}
Can you give two or more possible solutions $\mathbf{x}$ to the linear system above? Can you provide a closed form description of *all* possible solutions to this linear system?  


In [37]:
# Create our matrix A and vector y
# (feel free to replace by one of the matrices/vectors above)
A = np.array([[1,0,1], 
    [2,1,1]])

y = np.array([[1],[1]])

# Solve the linear system of equations
xest = np.linalg.lstsq(A,y,rcond=10)[0]
yest = np.matmul(A,xest)



print('A = ')
print(A)

print('\n y = ')
print(y)

print('\n xest = ')
print(xest)

print('\n A xest = ')
print(yest)

A = 
[[1 0 1]
 [2 1 1]]

 y = 
[[1]
 [1]]

 xest = 
[[ 0.33333333]
 [-0.33333333]
 [ 0.66666667]]

 A xest = 
[[1.]
 [1.]]


### Question: how does the LLS-solving function in the previous cell pick xest out of all the possible candidate solutions? 

Let's see if our matrix has a null space, and if so let's modify xest by adding a component within the null space

In [43]:
# Does our matrix have a null space? 
ns = slinalg.null_space(A)
print('Basis for null space of A: ')
print(ns)

# Generate alternative solution by adding something 
# within the nullspace of A to our current solution
xest2 = xest + np.matmul(ns,rnd.rand(1,ns.shape[1]))
print('\n xest2: ')
print(xest2)

# Check that our new solution still fits the data just as well
yest2 = np.matmul(A,xest2)
print('\n yest2: ')
print(yest2)




Basis for null space of A: 
[[-0.57735027]
 [ 0.57735027]
 [ 0.57735027]]

 xest2: 
[[-0.09344603]
 [ 0.09344603]
 [ 1.09344603]]

 yest2: 
[[1.]
 [1.]]


## Q1: A linear least squares problem with a square matrix A

Suppose we have a linear least-squares problem defined as $$\mathbf{x}^* = \arg \min_{\mathbf{x}} \| \mathbf{Ax} - \mathbf{y}\|^2$$

for some matrix $\mathbf{A} \in \mathbb{R}^{N \times N}$, some data vector $\mathbf{y} \in \mathbb{R}^{N}$, and some unknown vector $\mathbf{x} \in \mathbb{R}^{N}$. 

Is the following statement correct? Since we know that $\mathbf{A}$ is a square matrix (ie: there are as many unknowns as there are equations), we know that this LLS problem necessarily has a single unique solution $\mathbf{x}^*$ that perfectly fits the data, ie: $\| \mathbf{A \, x}^* - \mathbf{y}\|^2 = 0$. 

*Possible answers:*

A: Correct

B: Not correct. The system may have more than one solution, although we know that any solution will perfectly fit the data, ie: $\| \mathbf{A \, x}^* - \mathbf{y}\|^2 = 0$

C: Not correct. The system only has one solution, but this solution may not perfectly fit the data, ie: it is possible that $\| \mathbf{A \, x}^* - \mathbf{y}\|^2 > 0$

D: Not correct. The system may have multiple solutions, and these solutions may or may not fit the data perfectly. 

E: Not correct. This system has no solution, ever. 

## Q2: LLS solution

Using basic calculus, in order to minimize $ \| \mathbf{Ax} - \mathbf{y} \|_2^2 $ we can look for the choice of $\mathbf{x}$ (which we call $\mathbf{x}^*$) such that the gradient of $ \|  \mathbf{Ax} - \mathbf{y} \|_2^2 $ is zero, ie: 

\begin{equation}
   \mathbf{A}^T \left( \mathbf{A} \mathbf{x}^* - \mathbf{y} \right) = 0
\end{equation}
(let's stick to real-valued vectors and matrices, for simplicity), which can be rewritten as:
\begin{eqnarray}
\mathbf{A}^T \mathbf{A} \mathbf{x}^* = \mathbf{A}^T \mathbf{y}  
\end{eqnarray}
Now, if the matrix $\mathbf{A}^T \mathbf{A}$ has an inverse matrix $(\mathbf{A}^T \mathbf{A})^{-1}$, we can apply it to both sides of the equation above, ie:
\begin{eqnarray}
(\mathbf{A}^T \mathbf{A})^{-1} (\mathbf{A}^T \mathbf{A}) \mathbf{x}^* = (\mathbf{A}^T \mathbf{A})^{-1}\mathbf{A}^T \mathbf{y}  
\end{eqnarray}
or in other words,
\begin{equation} \label{eq:lls_closedsolution}
\mathbf{x}^* = (\mathbf{A}^T \mathbf{A})^{-1}\mathbf{A}^T \mathbf{y} 
\end{equation}
And this is our linear least squares solution. 

*Question:* Suppose we are given one data vector $\mathbf{y_a}$ and this leads to a solution $\mathbf{x}_a^*$ as described above. Now suppose we are given a different data vector $\mathbf{y_b} = 7 \cdot \mathbf{y_a}$. What will be the solution to our LLS problem with this new data vector $\mathbf{y_b}$?

A: The same, ie: $\mathbf{x}_b^* = \mathbf{x}_a^*$ 

B: The solution will change linearly with the data ie: $\mathbf{x}_b^* = 7 \cdot \mathbf{x}_a^*$ 

C: The solution will be all zeros

D: We don't have enough information to answer this question - it will depend on further specifics of $\mathbf{A}$ and $\mathbf{y_a}$.


## Q3: Weighted linear least squares

Now we want to elaborate on the previous example, still assuming that the matrix $\mathbf{A}^T \mathbf{A}$ has an inverse, as follows: we'd like to minimize $ \| \mathbf{W} ( \mathbf{Ax} - \mathbf{y}) \|_2^2 $ for some positive definite matrix $\mathbf{W}$. For example, let us assume that $ \mathbf{W}$ is a diagonal matrix where the entries are all strictly positive, ie: $W_{kk} > 0$, but these diagonal entries are generally different from each other. This weighted least squares formulation allows us to include information about which entries in our data vector we really want to fit closely (those entries $y_k$ with high corresponding $W_{kk}$ values), and which ones we do not care about as much (those entries $y_k$ with lower corresponding $W_{kk}$ values). 

Can you derive a closed form solution to our weighted LLS problem? 

A: Not in general.

B: Yes, we can just call $\mathbf{z} = \mathbf{W y}$ our new data vector, $\mathbf{B} = \mathbf{W A}$ our new system matrix, and our solution reduces to the LLS solution above, ie: 
\begin{equation} 
\mathbf{x}^* = (\mathbf{B}^T \mathbf{B})^{-1}\mathbf{B}^T \mathbf{z} 
\end{equation}
